# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
c_df = pd.read_csv('../output_data/Cities.csv')
c_df.head()


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,Bubaque,GW,78.75,81,100,1.54,11.2833,-15.8333,1626131101
1,Hermanus,ZA,46.80,66,100,27.00,-34.4187,19.2345,1626131104
2,Samarinda,ID,73.44,98,98,1.83,-0.5000,117.1500,1626131104
3,Airai,TL,58.35,54,13,3.65,-8.9266,125.4092,1626131104
4,Sitka,US,57.09,88,90,8.05,57.0531,-135.3300,1626131105


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)


cit = c_df[['Latitude', 'Longitude']]
humidity = c_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(cit, weights=humidity, dissipating=False, max_intensity=100,
                                 point_radius = 1))

fig

# #Set up your map
# fig = gmaps.figure()
# fig.add_layer(gmaps.heatmap_layer(city, weights=humidity))
# fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
fitweather = c_df.loc[(c_df['Temperature'] >= 70) & (c_df['Temperature'] <= 80) & 
                                  (c_df['Windspeed'] <= 20) & (c_df['Humidity'] <= 30)]
fitweather

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
48,Norman Wells,CA,79.48,24,10,6.24,65.2820,-126.8329,1626131121
139,Aklavik,CA,77.56,26,84,10.02,68.2191,-135.0107,1626131149
201,Murgab,TM,79.18,22,0,5.79,37.4966,61.9714,1626131167
259,High Level,CA,77.47,27,70,7.25,58.5169,-117.1360,1626131184
347,Fairbanks,US,79.02,30,1,0.00,64.8378,-147.7164,1626131190
515,Três Lagoas,BR,72.34,29,1,3.02,-20.7511,-51.6783,1626131031
523,Mayor Pablo Lagerenza,PY,78.94,27,27,6.15,-19.9309,-60.7718,1626131257
595,Zhetysay,KZ,71.98,24,1,7.02,40.7333,68.3000,1626131277
598,Korla,CN,77.22,28,72,9.22,41.7597,86.1469,1626131278
648,Birjand,IR,76.62,27,0,7.18,32.8663,59.2211,1626131292


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hlist = []

for i in range(len(fitweather)):
    lat = fitweather.loc[i]['Latitude']
    lng = fitweather.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hlist.append(jsn['results'][0]['name'])
    except:
        hlist.append("")
fitweather["Hotel Name"] = hlist
fitweather = fitweather.dropna(how='any')
fitweather.head()

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date,Hotel Name
0,Norman Wells,CA,79.48,24,10,6.24,65.2820,-126.8329,1626131121,Norman Wells
1,Aklavik,CA,77.56,26,84,10.02,68.2191,-135.0107,1626131149,Aklavik
2,Murgab,TM,79.18,22,0,5.79,37.4966,61.9714,1626131167,Mary
3,High Level,CA,77.47,27,70,7.25,58.5169,-117.1360,1626131184,High Level
4,Fairbanks,US,79.02,30,1,0.00,64.8378,-147.7164,1626131190,Fairbanks


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fitweather.iterrows()]
locations = fitweather[["Latitude", "Longitude"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))